# Web scraping completo

Si se ejecuta muchas veces seguidas da error por "exceso de redirecciones". Para solucionarlo hay que limpiar caché y borrar las cookies.

Periódicos incluidos: 
- El Confindencial (general)
- El Mundo (general)
- El País (general)
- ABC (general)
- El Economista (económico)
- Marca (deportivo)
- La Razón (general)
- Cinco Días (económico)
- As (deportivo)
- Expansión (económico)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import copy
import json

## El Confidencial

In [3]:
# URL de la que partimos
url = 'https://www.elconfidencial.com/espana/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")


# Comprobar cuál es la última página de noticias (como tiene varios números en el pie de página para seguir avanzando...)
last_page = int(soup.findAll('a',{'rel' : 'last'})[0]\
                .get('href')[:-1][soup.findAll('a',{'rel' : 'last'})[0].get('href')[:-1].rfind('/')+1:])


# Hacer lista con todas las url de cada página de noticias (elconfidencial/espana/2, elconfidencial/espana/3, etc.)
list_urls = [url] # la inicializamos con la propia home (elconfidencial/espana)
for i in range(2, last_page+1): # range 2 porque la 1 es la home
    list_urls.append(url + str(i) + '/')
   
# En ocasiones se generan errores al hacer scraping sobre todas las páginas de El Confidencial. Si es así, hay que 
# descomentar la siguiente línea y sacar solo las noticias de la primera página:

#list_urls = [url]

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "El Confidencial", para diferenciar este dataset de otros
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # para indicar si el artículo está en la cabecera o no. Binario (1 = sí está en cabecera, 0 = no)
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # para el número de página en el que está la noticia (entiendo que las que están en la pg 1 son más "importantes"
         #                                                     que las de la página 12, o al menos más recientes)
url = [] # para la url de la noticia, que sería la específica que el sistema le recomendaría (o no) al usuario


# Bucle para iterar sobre cada una de las páginas de noticias
for x, urll in enumerate(list_urls):
    i = 0
    peticion = requests.get(urll, allow_redirects = False)
    soup = BeautifulSoup(peticion.text, "lxml")

    # Para la noticia que esté en la cabecera - no todas tienen, por eso ponemos esta parte en "try"
    try:
        titulo.append(soup.findAll('a', attrs={'class': 'archive-article-top-link'})[0].get('data-title'))
        por.append(soup.findAll('span',{'class' : 'archive-article-top-author sig-color'})[0].text)
        cab.append(1) # 1 porque es la de la cabecera
        # Para comentarios: si no tiene, muestra 0. Si sí, muestra el número de comentarios
        if soup.findAll('div',{'class' : 'archive-header-top-author-box'})[0].find('em') == None:
            com.append(0)
        else:
            com.append(soup.findAll('div',{'class' : 'archive-header-top-author-box'})[0].find('em').text)
        
        pag.append(x+1) # x se saca del inicio del bucle, por "enumerate", como comienza en 0, se le suma 1 al índice
        
        # Para la url de cada noticia
        url_noticia = soup.findAll('a', attrs={'class': 'archive-article-top-link'})[0].get('href')
        url.append(url_noticia)
        peticion_noticia = requests.get(url_noticia, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
        texto.append(soup_noticia.findAll('div', attrs={'class': 'news-body-center cms-format'})[0].text)
        
        web.append('El Confidencial')
    except:
        pass

    # Para el resto de noticias, las que no están en cabecera
    # Bucle desde 0 hasta la cantidad total de noticias que haya en cada una de las páginas
    
    for i in range(0,len(soup.findAll('a', attrs={'href': re.compile("^https://"), 'class': 'archive-article-link'}))):
        web.append('El Confidencial')
        titulo.append(soup.findAll('a', attrs={'href': re.compile("^https://"), 'class': 'archive-article-link'})[i].get('title'))
        por.append(soup.findAll('span',{'class' : 'archive-article-author sig-color'})[i].text)
        cab.append(0) # 0 porque no son noticias de cabecera
        # Para comentarios: si no tiene, muestra 0. Si sí, muestra el número de comentarios
        if soup.findAll('div',{'class' : 'archive-article-author-box'})[i].find('em') == None:
            com.append(0)
        else:
            com.append(soup.findAll('div',{'class' : 'archive-article-author-box'})[i].find('em').text)
        pag.append(x+1)
        
        # Para la url de cada noticia
        url_noticia = soup.findAll('a', attrs={'href': re.compile("^https://"), 'class': 'archive-article-link'})[i].get('href')
        url.append(url_noticia)
        peticion_noticia = requests.get(url_noticia, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
        
        # Como el cuerpo de la noticia está en formatos diferentes en función de cada noticia:
        try:
            texto.append(soup_noticia.findAll('div', attrs={'class': 'news-body-center cms-format'})[0].text)
        except:
            try:
                texto.append(soup_noticia.findAll('div', attrs={'class': 'newsType__content'})[0].text)
            except:
                texto.append('Revisar') # hay noticias que no tienen texto, son solo vídeos o cosas interactivas

# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe de Pandas

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}

ec = pd.DataFrame(table_dict) # "ec" = El Confidencial

# Para sacar la fecha de la noticia desde la url de la misma (yyyy-mm-dd):
ec['fecha1'] = ec['URL'].str.split('/').str[4] # divide la url por cada "/" que encuentre y nos quedamos con el de la posición 4
ec['fecha2'] = ec['URL'].str.split('/').str[5] # divide la url por cada "/" que encuentre y nos quedamos con el de la posición 5
ec['fecha3'] = ec['URL'].str.split('/').str[6] # divide la url por cada "/" que encuentre y nos quedamos con el de la posición 6
ec['fecha4'] = ec['fecha1'].apply(lambda x: len([s for s in x if s.isdigit()])) # cuenta los números que hay en este campo
ec['fecha5'] = ec['fecha2'].apply(lambda x: len([s for s in x if s.isdigit()]))
ec['fecha6'] = ec['fecha3'].apply(lambda x: len([s for s in x if s.isdigit()]))

# Función personalizada para sacar cuál es la fecha
def fecha(row):
    if row['fecha4'] == 8: # para que sea una fecha, debe tener 8 números (yyyy-mm-dd)
        return row['fecha1']
    if row['fecha5'] == 8: # si el primer campo no es una fecha, prueba con el segundo
        return row['fecha2']
    if row['fecha6'] == 8: # si el segundo campo no es una fecha, prueba con el tercero
        return row['fecha3']
    else:
        return 'Otra'
    
ec['Fecha'] = ec.apply (lambda row: fecha(row), axis=1) # aplicamos la función anterior a la columna "Fecha" para sacar la 
                                                        # fecha definitiva

# Borramos las columnas auxiliares
del ec['fecha1']
del ec['fecha2']
del ec['fecha3']
del ec['fecha4']
del ec['fecha5']
del ec['fecha6']


## El Mundo

In [4]:
# URL de la que partimos
url = 'https://www.elmundo.es/espana.html'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "El Mundo", para diferenciar este dataset de otros
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # indicador de cabecer
aux = [] # auxiliar para los comentarios
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # número de página en el que está la noticia
url = [] # para la url de la noticia

# Para sacar los autores
nested_lst = soup.findAll('div', {"class" : 'ue-c-cover-content__list-inline'})
lista = []
for i in range(0, len(nested_lst)):
    lista.append(nested_lst[i].text)

for l in range(0, len(lista)):
    result = re.search('\n(.*)\n', lista[l])
    if result is None: 
        por.append('-')
    else:
        por.append(result.group(1))

# Para sacar los comentarios  
for i in range(0, len(lista)):
    aux.append(lista[i].rsplit('\n', 1)[-1])

for c in range(0, len(aux)):
    prev = aux[c].split("}})")
    if len(prev) == 1:
        com.append(prev[0].split(' ')[0])
    else:
        com.append(prev[-1].split(' ')[0])

# Resto: título, cabecera siempre 0, página siempre 1
for i in range(0,len(soup.findAll('h2', {"class" : 'ue-c-cover-content__headline'}))):
    web.append('El Mundo')
    
    try:
        titulo.append(
            soup.findAll('div', {"class" : 'ue-c-cover-content__body'})[i].find('span', {'class' : 'ue-c-cover-content__kicker'}).text\
                  + soup.findAll('h2', {"class" : 'ue-c-cover-content__headline'})[i].text)
    except:
        titulo.append(soup.findAll('h2', {"class" : 'ue-c-cover-content__headline'})[i].text)
        
    # Cabecera
    if i == 0:
        cab.append(1) 
    else:
        cab.append(0)
    
    pag.append(1) # Está todo en la misma página, no hay varias
    
    # Para la url
    url_noticia = soup.findAll('a', attrs={'href': re.compile("^https://"), 'class': 'ue-c-cover-content__link'})[i].get('href')
    url.append(url_noticia)
    peticion_noticia = requests.get(url_noticia, allow_redirects = False)
    soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
    
    
    # Para el texto de la noticia. Como hay diferentes formatos, los meto en distintos "try"
    # EL MUNDO ES DE PAGO, POR LO QUE SOLO EXTRAE LA PARTE GRATUITA DE LA NOTICIA (1 párrafo, generalmente)
    try:
        #texto.append(soup_noticia.findAll('p', {'class': 'ue-c-article--first-letter-highlighted'})[0].text)
        raw = []
        for elm in soup_noticia.select('p'):
            if elm.find('span'):
                    continue
            raw.append(elm)

        raw2 = str(raw).split('<p class="ue-c-article__standfirst">')[1].split('<p class="ue-c-newsletter-widget__subline">')[0]
        texto.append(re.sub('<[^>]+>', '', raw2))
        
    except:
        try:
            texto.append(soup_noticia.findAll('section', {'class': 'cuerpo'})[0].text)
        except:
            try:
                texto.append(soup_noticia.findAll('p', {'class': 'ue-c-article--first-letter-highlighted'})[0].text)
            except:
                try:
                    tex = []

                    for i in range(0, len(soup.findAll('p', {'class': ''}))):
                        tex.append(soup.findAll('p', {'class': ''})[i].text)

                    tex2 = ' '.join(tex)
                    texto.append(tex2)
                except:
                    try:
                        texto.append(soup_noticia.findAll('dl')[0].text)
                    except:
                        texto.append(None)
            

    
# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
em = pd.DataFrame(table_dict)

# Para sacar la fecha desde la url (yyyy-mm-dd)
em['Fecha'] = em['URL'].str.split('/').str[4] + '-' + em['URL'].str.split('/').str[5] + '-' + em['URL'].str.split('/').str[6]


## El País

No se puede sacar el número de comentarios con BeautifulSoup, se podría sacar con Selenium/webdriver en caso de que fuese necesario.

In [5]:
# URL de la que partimos
url = 'https://elpais.com'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "El País"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # indicador de cabecera
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # número de página en el que está la noticia
url = [] # para la url de la noticia

for i in range(0, len(soup.findAll('article'))):
    web.append('El País') # Siempre el mismo
    #titulo.append(soup.findAll('article')[i].find('h2').text)
    tit = soup.findAll('article')[i].find('h2').text
    titulo.append(tit)
    
    # Autor
    try:
        aut = []
        for j in range(0, len(soup.findAll('article')[i].findAll('a', {'class' : 'author'}))):
            aut.append(soup.findAll('article')[i].findAll('a', {'class' : 'author'})[j].text)

        autt = list(aut)
        por.append(' '.join(autt))
    except:
        por.append('Revisar')
    
    # Cabecera: 1 si es la primera noticia, 0 el resto
    if i == 0:
        cab.append(1)
    else: 
        cab.append(0)
     
    com.append(None) # No se puede sacar con BeautifulSoup. Se podría con Selenium / Webdriver
    pag.append(1) # Están todas las noticias en la home
    
    # URL
    
    try:
        prev = soup.findAll('article')[i].findAll('a', {'class' : False})
        if 'por=mosaico' in prev[0].get('href'):
            url_aux = prev[1].get('href')
        else:
            url_aux = prev[0].get('href')

        if url_aux[0] == '/':
            urll = 'https://elpais.com' + url_aux
        else:
            urll = url_aux

        url.append(urll)
    except:
        urll = None
        url.append(urll)
    
    try:
        if 'huffingtonpost' not in urll: # la web de Huffingtonpost da errores 
            # Texto
            peticion_noticia = requests.get(urll, allow_redirects = False)
            soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

            # Como el cuerpo de la noticia está en formatos diferentes en función de cada noticia:
            try:
                texto.append(soup_noticia.findAll('div', attrs={'class': 'a_b article_body | color_gray_dark'})[0].text)
            except:
                texto.append(None) # hay noticias que no tienen texto, son solo vídeos o cosas interactivas

        else:
            #texto.append('Hufftington')
            texto.append(tit)
    except:
        texto.append(None)
    
# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
ep = pd.DataFrame(table_dict)


# Para sacar la fecha de la noticia desde la url de la misma (yyyy-mm-dd):
r = re.compile("\d{4}-\d{2}-\d{2}")
r2 = re.compile("\d{4}/\d{2}/\d{2}")

def match(x):
    m = r.search(x)
    m2 = r2.search(x)
    if m:
        return  m.group()
    elif m2:
        return m2.group()
    return  ""

try:
    ep['Fecha'] = ep["URL"].apply(match) # aplicamos la función anterior a la columna "URL" para sacar la 
                                                        # fecha definitiva
    ep['Fecha'] = ep['Fecha'].str.replace('/','-')
except:
    pass

## ABC

In [6]:
# URL de la que partimos
url = 'https://www.abc.es/'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "ABC"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # indicador de cabecera
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # número de página en el que está la noticia
url = [] # para la url de la noticia

for i in range(0, len(soup.findAll('span', {'class' : 'textos'}))):
    web.append('ABC')
    
    # Titular
    try:
        titulo.append(soup.findAll('span', {'class' : 'textos'})[i].find('a').get('title'))
    except:
        titulo.append('Revisar')
    
    # Autor
    try:
        aux = soup.findAll('span', {'class' : 'textos'})[i].find('footer').find('a', {'class' : 'autor'}).text
    except:
        try:
            aux = soup.findAll('span', {'class' : 'textos'})[i].find('footer').find('span').text
        except:
            aux = 'Revisar'
    if aux == 'Comentar':
        por.append(None)
    else:
        por.append(aux)

    # Cabecera
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
    
    
    com.append(None) # Incapaz de sacarlo con BeautifulSoup
    pag.append(1)
    
    # URL
    try:
        urll = soup.findAll('span', {'class' : 'textos'})[i].find('a').get('href')
        if urll[0] == '/':
            url2 = 'https://www.abc.es' + urll
        elif urll[0] == 't':
            url2 = 'h' + urll
        else:    
            url2 = urll
        url.append(url2)
    except:
        url.append('Revisar')
    
    # Texto
    peticion_noticia = requests.get(url2, allow_redirects = False)
    soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
    
    tex = []
    for i in range(0, len(soup_noticia.findAll('p', {'class' : ''})[1:])):
        tex.append(soup_noticia.findAll('p', {'class' : ''})[1:][i].text)
    texto.append(' '.join(tex))

    
    
# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}
abc = pd.DataFrame(table_dict)


# Fecha
r = re.compile("\d{4}\d{2}\d{2}")

def match(x):
    m = r.search(x)
    if m:
        return  m.group()
    return  ""

abc['Fecha'] = abc["URL"].apply(match) # aplicamos la función anterior a la columna "URL" 

abc['Fecha'] = abc['Fecha'].apply(lambda s: "{}-{}-{}".format(s[0:4], s[4:6], s[6:]))
abc['Fecha'] = abc['Fecha'].replace('//', None)

## El Economista

In [7]:
# URL de la que partimos
url = 'https://www.eleconomista.es/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "El Economista"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # para indicar si el artículo está en la cabecera o no. Binario (1 = sí está en cabecera, 0 = no)
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # para el número de página 
url = [] # para la url de la noticia, que sería la específica que el sistema le recomendaría (o no) al usuario
fecha = []

for i in range(0, len(soup.findAll('div', {'class' : 'article'}))):
    web.append('El Economista')
    
    # Titular
    try:
        titulo.append(soup.findAll('div', {'class' : 'article'})[i].find('h2').text)
    except:
        titulo.append(soup.findAll('div', {'class' : 'article'})[i].find('span').text)
    
    # Cabecera = 1 si es la primera noticia, 0 para el resto
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
    
    # Página = 1 siempre
    pag.append(1)
    
    # URL
    try:
        urll = soup.findAll('div', {'class' : 'article'})[i].find('h2').find('a').get('href')
        if urll[0] == '/':
            urll = 'https:' + urll
        url.append(urll)
    except:
        try:
            urll = soup.findAll('div', {'class' : 'article'})[i].find('span').find('a').get('href')
            if urll[0] == '/':
                urll = 'https:' + urll
            url.append(urll)
        except:
            try:
                urll = soup.findAll('div', {'class' : 'article'})[i].find('a').get('href')
                if urll[0] == '/':
                    urll = 'https:' + urll
                url.append(urll)
            except:
                urll = 'Revisar'
                url.append(urll)
    
    # Texto
    #texto.append('Revisar')
    try:
        peticion_noticia = requests.get(urll, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
        tex = soup_noticia.findAll('meta', property = 'og:description')[0]['content']
        texto.append(tex)
    except:
        texto.append('Revisar')
    
    # Autor
    try:
        por.append(soup.findAll('div', {'class' : 'article'})[i].find('li', {'class' : 'articleAuthor'}).text)
    except:
        try:
            aut = []
            for a in range(0, len(soup_noticia.findAll('meta', itemprop = 'author'))):
                aut.append(soup_noticia.findAll('meta', itemprop = 'author')[a]['content'])
            aut = list(set(aut))
            por.append(aut)
        except:
            por.append('Revisar')
    
    # Comentarios - formato JSON
    try:
        com.append(soup.findAll('div', {'class' : 'article'})[i].find('ul').findAll('li', {'class' : 'articleComments'})[0].find('span').text)
    except:
        try:
            json_data = soup_noticia.findAll('script', text=re.compile("noticiaComentarios"))
            json_str = str(json_data).split("noticiaComentarios")[1].replace("'", "")
            json_com = [int(s) for s in json_str.split() if s.isdigit()][0]
            com.append(json_com)
        except:
            com.append(None)
    
    # Fecha 
    try:
        json_data = soup_noticia.findAll('script', text=re.compile("noticiaComentarios"))
        json_str = str(json_data).split('noticiaFechaPublicacion')[1].split('noticiaComentarios')[0].replace("'", "").replace(',','')
        aux = [int(s) for s in json_str.split() if s.isdigit()]
        fecha.append(str(datetime.date(datetime.strptime(str(aux[0])[0:8], '%Y%m%d'))))
    except:
        fecha.append(None)
        
# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe de Pandas

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url,\
              'Fecha': fecha}

ee = pd.DataFrame(table_dict) 
ee['Comentarios'] = ee['Comentarios'].replace('', 0)

## Marca

In [8]:
url = 'https://www.marca.com/'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "Marca"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # indicador de cabecera
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # número de página en el que está la noticia
url = [] # para la url de la noticia
fecha = []

for i in range(0, len(soup.findAll('article'))):
    web.append('Marca')
        
    # Cabecera
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
        
    # Comentarios
    try:
        com.append(soup.findAll('article')[i].findAll('span', {'class' : 'number-comments'})[0].text)
    except:
        com.append(None)
    
    # Página
    pag.append(1)
    
    # URL
    try:
        #urll = soup.findAll('article')[i].findAll('a', {'itemprop' : 'url'})[0].get('href')
        urll = soup.findAll('article')[i].findAll('h2')[0].find('a').get('href')
        url.append(urll)
    except:
        urll = None
        url.append(None)
    
    # Texto
    try:
        peticion_noticia = requests.get(urll, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

        tex = []
        for j in range(0, len(soup_noticia.findAll('div', {'class' : 'row content cols-30-70'})[0].findAll('p', {'class' : ''}))):
            tex.append(soup_noticia.findAll('div', {'class' : 'row content cols-30-70'})[0].findAll('p', {'class' : ''})[j].text)
        texto.append(' '.join(tex))
    except:
        texto.append(None)

     # Autores
    try:
        por.append(soup.findAll('article')[i].findAll('ul', {'class' : 'mod-author'})[0].find('span').text)
    except:
        try:
            por.append(soup_noticia.findAll('ul', {'class' : 'author'})[0].text)
        except:
            por.append(None)
    
    # Titular
    try:
        #titulo.append(soup.findAll('article')[i].findAll('a', attrs={'itemprop': 'url'})[0].text)
        titulo.append(soup.findAll('article')[i].findAll('h2')[0].text)
    except:
        try:
            titulo.append(soup_noticia.findAll('div', {'class' : 'titles'})[0].findAll('h1')[0].text)
        except:
            titulo.append(None)
    
    # Fecha
    # Para sacar la fecha de la noticia desde la url de la misma (yyyy-mm-dd):
    r = re.compile("\d{4}/\d{2}/\d{2}")
    r2 = re.compile("\d{4}-\d{2}-\d{2}")

    def match(x):
        m = r.search(x)
        m2 = r2.search(x)
        if m:
            return  m.group()
        elif m2:
            return m2.group()
        return  ""

    try:
        aux = match(urll).replace('/','-')
        fecha.append(aux)
    except:
        fecha.append(None)
    
# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url,\
              'Fecha': fecha}

m = pd.DataFrame(table_dict)


## La Razón

Este diario contiene la información en formato JSON.

In [9]:
# URL de la que partimos
url = 'https://www.larazon.es/'
peticion = requests.get(url)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "La Razón"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # indicador de cabecera
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # número de página en el que está la noticia
url = [] # para la url de la noticia
fecha = []

counter = 0 # auxiliar para detectar cuál es la primera noticia (y así asignarla a cabecera)

for i in range(0, len(soup.select("[type='application/ld+json']"))):
   
    data = soup.select("[type='application/ld+json']")[i]
    
    if json.loads(data.text) != []:
        
        counter += 1
        web.append('La Razón')
        
        # Titular
        json_headline = json.loads(data.text)['headline']
        titulo.append(json_headline)
        
        # Fecha
        json_date = json.loads(data.text)['datePublished'][:10]
        fecha.append(json_date)
        
        # Autores
        autores = []
        for j in range(0, len(json.loads(data.text)['author'][0])):
            autores.append(json.loads(data.text)['author'][0][j]['name'])
        por.append(', '.join(autores))
        
        # Comentarios: no hay sección de comentarios de cada noticia, asignamos valor nulo
        com.append(None)
        
        # Página siempre 1, están todas las noticias en la home
        pag.append(1)
        
        # Cabecera
        if counter == 1:
            cab.append(1)
        else:
            cab.append(0)
        
        # URL
        json_url = json.loads(data.text)['url']
        url.append(json_url)
        
        # Texto
        peticion_noticia = requests.get(json_url)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

        data_noticia = soup_noticia.select("[type='application/ld+json']")[0]
        try: # Si tiene descripción y cuerpo de noticia
            noticia = [json.loads(data_noticia.text)[0]['description'], json.loads(data_noticia.text)[0]['articleBody']]
        except:
            try: # si solo tiene cuerpo de noticia
                noticia = [json.loads(data_noticia.text)[0]['articleBody']]
            except:
                noticia = ['Revisar']

        texto.append(' '.join(noticia))
        
    else:
        continue


# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe
table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url,
              'Fecha' : fecha}
lr = pd.DataFrame(table_dict)

## Cinco Días

In [10]:
# URL de la que partimos
url = 'https://cincodias.elpais.com/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "Cinco Días"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # para indicar si el artículo está en la cabecera o no. Binario (1 = sí está en cabecera, 0 = no)
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # para el número de página en el que está la noticia 
url = [] # para la url de la noticia, que sería la específica que el sistema le recomendaría (o no) al usuario
fecha = []

for i in range(0, len(soup.findAll('article'))):
    web.append('Cinco Días')
    
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
    
    pag.append(1)
    
    # Titular
    try:
        titulo.append(re.findall(r'\n(.*)\n', soup.findAll('article')[i].find('h2').text)[0])
    except:
        titulo.append(soup.findAll('article')[i].find('h2').text)
        
    # Autor
    try:
        por.append(soup.findAll('article')[i].find('span', {'class' : 'autor-nombre'}).text)
    except:
        por.append(None)
        
    # Comentarios: None. No hay en esta página
    com.append(None) 
    
    # URL
    urll = soup.findAll('article')[i].find('h2').find('a').get('href')
    
    if urll[0:2] == '/c':
        urll = 'https://cincodias.elpais.com' + urll
    elif urll[0:2] == '//':
        urll = 'https:' + urll
    elif urll[0:2] == '/r':
        urll = 'https://retina.elpais.com/' + urll
    else:
        urll = urll
    
    url.append(urll)
    
    # Texto
    
    if urll[8:14] != 'retina':
        try:
            peticion_noticia = requests.get(urll, allow_redirects = False)
            soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

            try:
                h1 = soup_noticia.findAll('article')[0].findAll('h1')[0].text
            except:
                h1 = ''

            try:
                h2 = soup_noticia.findAll('article')[0].findAll('h2')[0].text
            except:
                h2 = ''

            cuerpo = re.findall(r'(.*)\n', soup_noticia.findAll('article')[0].find('div', {'class' : 'articulo-cuerpo'}).text)

            tex = [h1, h2, ' '.join(cuerpo)]
            tex = ' '.join(tex)
            texto.append(tex)
        except:
            texto.append('Revisar')
    else:
        try:
            peticion_noticia = requests.get(urll, allow_redirects = False)
            soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

            cuerpo = soup_noticia.findAll('div', {'class' : 'articulo-cuerpo'})[0].text.replace('\n', '')
            texto.append(cuerpo)
        except:
            texto.append('Revisar2')

    
    # Fecha
    fecha.append(None)
    

# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe de Pandas

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url,\
              'Fecha': fecha}


cd = pd.DataFrame(table_dict) 

# Fecha
r = re.compile(r'(\d+/\d+/\d+)')

def match(x):
    m = r.search(x)
    if m:
        return  m.group()
    return  ""  

cd['Fecha'] = cd["URL"].apply(match) # aplicamos la función anterior a la columna "URL" 
cd['Fecha'] = cd['Fecha'].apply(lambda s: "{}-{}-{}".format(s[0:4], s[5:7], s[8:]))

cd = cd[cd['Texto'] != 'Revisar']

## As

In [11]:
# URL de la que partimos
url = 'https://as.com/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "As"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # para indicar si el artículo está en la cabecera o no. Binario (1 = sí está en cabecera, 0 = no)
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # para el número de página en el que está la noticia 
url = [] # para la url de la noticia, que sería la específica que el sistema le recomendaría (o no) al usuario
fecha = []


for i in range(0, len(soup.findAll('article'))):
    web.append('As')
    
    # Cabecera
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
        
    # Página siempre 1
    pag.append(1)
    
    # Comentarios
    try:
        com.append(soup.findAll('article')[i].find('span', {'class' : 'comment-n'}).text)
    except:
        com.append(None)
    
    # URL
    try:
        urll = soup.findAll('article')[i].find('h2').find('a').get('href')
    except:
        try:
            urll = soup.findAll('article')[i].find('h3').find('a').get('href')
        except:
            try:
                urll = soup.findAll('article')[i].find('h4').find('a').get('href')
            except:
                urll = 'Revisar'
    
    url.append(urll)
    
    # Título
    try:
        titulo.append(re.findall(r'\n(.*)\n', soup.findAll('article')[i].find('h2').text)[0])
    except:
        try:
            titulo.append(re.findall(r'\n(.*)\n', soup.findAll('article')[i].find('h3').text)[0])
        except:
            try:
                titulo.append(re.findall(r'\n(.*)\n', soup.findAll('article')[i].find('h4').text)[0])
            except:
                titulo.append('Revisar')
    
    # Autor
    try:
        por.append(re.findall(r'\n(.*)\n', soup.findAll('article')[i].find('span', {'class' : "autor-nombre"}).text)[0])
    except:
        por.append(None)
    
    # Texto
    try:
        peticion_noticia = requests.get(urll, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")
    
        try:
            texto.append(soup_noticia.findAll('div', {'id' : 'cuerpo_noticia'})[0].text.replace('\n', '').replace('\t', ''))
        except:
            try:
                texto.append(soup_noticia.findAll('div', {'class' : 'stream-ev-wr'})[0].text.replace('\n', '').replace('\t', ''))
            except:
                texto.append(None)
    except:
        texto.append(None)
    
    # Fecha
    try:
        fecha.append(soup.findAll('article')[i].find('span', {'class' : 'fecha'}).text)
    except:
        fecha.append(None)


table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url,\
              'Fecha': fecha}


df_as = pd.DataFrame(table_dict) 
    
df_as['Fecha'] = pd.to_datetime(df_as['Fecha']).dt.strftime('%Y-%m-%d')

df_as = df_as.dropna(how='all')

## Expansión

In [12]:
# URL de la que partimos
url = 'https://www.expansion.com/'
peticion = requests.get(url, allow_redirects = False)
soup = BeautifulSoup(peticion.text, "lxml")

# Inicializamos listas vacías para llenarlas con bucles
web = [] # será siempre "Expansión"
titulo = [] # para el título de la noticia
por = [] # para el autor 
cab = [] # para indicar si el artículo está en la cabecera o no. Binario (1 = sí está en cabecera, 0 = no)
com = [] # para el número de comentarios que tiene
texto = [] # para el texto de la noticia
pag = [] # para el número de página en el que está la noticia
url = [] # para la url de la noticia, que sería la específica que el sistema le recomendaría (o no) al usuario
fecha = []

for i in range(0, len(soup.findAll('article'))):
    web.append('Expansión')
    
    # Titular
    titulo.append(soup.findAll('article')[i].find('h2').text)
    
    # Autor
    try:
        por.append(soup.findAll('article')[i].find('span', {'class' : 'ue-c-cover-content__byline-name'}).text.replace('\n', '').replace('Redacción: ', ''))
    except:
        por.append(None)
    
    # Cabecera
    if i == 0:
        cab.append(1)
    else:
        cab.append(0)
        
    # Página
    pag.append(1)
    
    # Comentarios: no hay
    com.append(None)
    
    # URL
    urll = soup.findAll('article')[i].find('h2').find('a').get('href')
    url.append(urll)
    
    # Texto
    
    try:
        peticion_noticia = requests.get(urll, allow_redirects = False)
        soup_noticia = BeautifulSoup(peticion_noticia.text, "lxml")

        try:
            try:
                stand_first = soup_noticia.findAll('p', {'class' : 'ue-c-article__standfirst'})[0].text
            except:
                stand_first = None
            
            tex = []
            for i in range(0, len(soup_noticia.findAll('div', {'class' : 'ue-c-article__body'})[0].findAll('p', {'class' : ''}))):
                tex.append(soup_noticia.findAll('div', {'class' : 'ue-c-article__body'})[0].findAll('p', {'class' : ''})[i].text)

            tex = ' '.join(tex)
            tex_prev = [stand_first, tex]

            texto.append(' '.join(tex_prev))
        except:
            texto.append(None)
    except:
        texto.append(None)
    
    # Fecha
    fecha.append('Revisar')


# Hacemos un diccionario con todos los resultados obtenidos y lo pasamos a formato dataframe de Pandas

table_dict = {'Web': web,\
              'Título': titulo,\
              'Autor': por,\
              'Cabecera': cab,\
              'Comentarios': com,\
              'Texto': texto,\
              'Página' : pag,\
              'URL': url}

ex = pd.DataFrame(table_dict) 

# Fecha
r = re.compile(r'(\d+/\d+/\d+)')

def match(x):
    m = r.search(x)
    if m:
        return  m.group()
    return  ""  

ex['Fecha'] = ex["URL"].apply(match) # aplicamos la función anterior a la columna "URL" 
ex['Fecha'] = ex['Fecha'].apply(lambda s: "{}-{}-{}".format(s[0:4], s[5:7], s[8:]))

## Dataframe conjunto

Dataframe uniendo todas las noticias anteriores.  
Saca algunas líneas "feas" adicionales, hay que hacer limpieza antes de intentar crear algún algoritmo, igual que al final del texto de la noticia también añade cosas adicionales (ver enlaces de interés, suscríbete, gracias por leer, etc.)

In [13]:
# Lista con los dataframes que tenemos de los diferentes periódicos
news = [ec, em, ep, abc, ee, m, lr, cd, df_as, ex]

# Los unimos, y para que no se repitan los índices, los reseteamos
df = pd.concat(news).reset_index(drop=True)

# Para crear el id (único) de cada noticia

generales = ['El Confidencial', 'El Mundo', 'El País', 'ABC', 'La Razón']
economicos = ['El Economista', 'Cinco Días', 'Expansión']
deportivos = ['Marca', 'As']

def tipo_diario(row):
    if row['Web'] in generales:
        val = 'gen_' 
    elif row['Web'] in economicos:
        val = 'eco_'
    elif row['Web'] in deportivos:
        val = 'dep_' 
    else:
        val = -1
    return val

df['new_id'] = df.apply(tipo_diario, axis=1)
indice = (df.index + 1).astype(str)
df['new_id'] = df['new_id'] + indice

# Poner la columna de ID en la primera posición
new_id = df['new_id']
df.drop(labels=['new_id'], axis=1, inplace = True)
df.insert(0, 'new_id', new_id)

# Ver el dataframe final
df

,new_id,Web,Título,Autor,Cabecera,Comentarios,Texto,Página,URL,Fecha
0,gen_1,El Confidencial,Los alcaldes aplazan la decisión de la hucha d...,EFE,1,0,31/07/2020 12:06 - ...,1,https://www.elconfidencial.com/espana/2020-07-...,2020-07-31
1,gen_2,El Confidencial,Sánchez concede a Urkullu 2.000 millones más p...,A. Pérez Giménez,0,22,31/07/2020 11:00 - ...,1,https://www.elconfidencial.com/espana/2020-07-...,2020-07-31
2,gen_3,El Confidencial,Interior compra 1.300 'mascarillas chic' solo ...,Roberto R. Ballesteros,0,5,31/07/2020 05:00Adelantado en La Dirección Gen...,1,https://www.elconfidencial.com/espana/2020-07-...,2020-07-31
3,gen_4,El Confidencial,Sanidad dice que es público el comité que Ribe...,Darío Ojeda,0,5,31/07/2020 05:00Adelantado en Tres meses despu...,1,https://www.elconfidencial.com/espana/2020-07-...,2020-07-31
4,gen_5,El Confidencial,"El 50,5% de los catalanes rechaza la independe...",Europa Press,0,0,31/07/2020 12:20 - ...,1,https://www.elconfidencial.com/espana/cataluna...,2020-07-31
...,...,...,...,...,...,...,...,...,...,...
864,eco_865,Expansión,El déficit del Estado se quintuplica hasta 48....,None,0,None,El déficit del Estado alcanzó en el primer sem...,1,https://www.expansion.com/economia/2020/07/30/...,2020-07-30
865,eco_866,Expansión,El rover Perseverance de la NASA parte rumbo a...,None,0,None,None,1,https://videos.expansion.com/v/0_udydlkib-el-r...,--
866,eco_867,Expansión,Videoanálisis técnico: Precaución en los 7.040...,ROBERTO MO...,0,None,None,1,https://videos.expansion.com/v/0_wyr4zvsq-vide...,--
867,eco_868,Expansión,Esto es lo que pasa si no lavas o cambias la m...,None,0,None,None,1,https://videos.expansion.com/v/0_kiw8z5bi-esto...,--


In [14]:
len(df)

869

In [15]:
# Exportar a csv
nombre = 'news_completo_' + datetime.today().strftime('%d%m%Y') + '.csv'
df.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')

# Clientes

5 modalidades diferentes de suscripción:  

- Alpha: suscripción básica (diarios generalistas)
- Beta: diarios generalistas + especializados en deportes
- Gamma: diarios generalistas + especializados en economía/finanzas
- Delta: suscripción "familiar", incluye la Omega x 4 usuarios
- Omega: suscripción completa (diarios generalistas + deportes + economía/finanzas)

### Dataframe de LECTURAS

Tras obtener el dataframe completo con todas las noticias (df), creamos otro dataframe con 45.000 clientes, indicando cuáles de esas noticias han leído (1) y cuáles no (0). 
Para que todos los usuarios no sean iguales, creamos 9 perfiles diferentes, con distinta probabilidad de que lean una noticia o no (de forma aleatoria), para tener clientes que leen mucho y otros que leen poco.
Generamos cada "sub-dataframe" en función de cada una de las diferentes suscripciones que se ofrecen, ya que por ejemplo las noticias de diarios deportivos no estarán disponibles para la suscripción Alpha (diarios generales).

In [16]:
np.random.seed(42)

# Las columnas del dataframe serán todas las noticias que provienen del df de noticias
columnas = df['new_id']

# Asumimos reparto equitativo de los clientes entre las 5 suscripciones
prob = [0.2, 0.2, 0.2, 0.2, 0.2]
suscripciones = ['alpha', 'beta', 'gamma', 'delta', 'omega']

# Creamos 45.000 clientes, en principio con números aleatorios para todas las noticias
clientes_0 = pd.DataFrame(np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(45000, len(columnas))), columns = columnas)

# 9.000 en cada segmento
clientes_alpha, clientes_beta, clientes_gamma, clientes_delta, clientes_omega = np.split(clientes_0, 5)

### ALPHA: suscripción a diarios generalistas

In [17]:
# Columnas del dataframe de noticias que aplican a este grupo; solo las generales
cols_alpha = [col for col in clientes_0 if col.startswith('gen')]

cols = clientes_0.columns.values

# Máscara para identificar las noticias que están disponibles en esta suscripción
mask_alpha = []

for i in range(0, len(cols)):
    mask_alpha.append(cols[i] in cols_alpha)

# Máscara para identificar las noticias que NO están disponibles en esta suscripción
mask_no_alpha = [not i for i in mask_alpha]

# Las noticias que no están disponibles se quedan en nulo, para diferenciarlas de las que sí están disponibles pero no se han 
# leído (0)
clientes_alpha[clientes_alpha.columns[mask_no_alpha]] = None

# Creamos dataframe de clientes_alpha
ca = clientes_alpha[clientes_alpha.columns[mask_alpha]].copy()

# Diferentes perfiles
ca.loc[0:999,] = np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[1000:1999,] = np.random.choice(np.arange(0, 2), p=[0.2, 0.8], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[2000:2999,] = np.random.choice(np.arange(0, 2), p=[0.3, 0.7], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[3000:3999,] = np.random.choice(np.arange(0, 2), p=[0.4, 0.6], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[4000:4999,] = np.random.choice(np.arange(0, 2), p=[0.5, 0.5], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[5000:5999,] = np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[6000:6999,] = np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[7000:7999,] = np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 
ca.loc[8000:8999,] = np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=(1000, len(clientes_alpha.columns[mask_alpha]))) 

ca.insert(0, 'suscripcion', 'alpha')
ca

new_id,suscripcion,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,...,gen_574,gen_575,gen_576,gen_577,gen_578,gen_579,gen_580,gen_581,gen_582,gen_583
0,alpha,1,1,1,0,0,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
1,alpha,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
2,alpha,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,alpha,1,1,1,0,1,1,1,0,1,...,1,1,1,1,0,1,1,1,1,1
4,alpha,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,alpha,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8996,alpha,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8997,alpha,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8998,alpha,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,1,0,0,0,0


In [18]:
# Comprobación de que leen diferente candiad de noticias
ca.sum(axis=1)

0       393
1       397
2       402
3       405
4       402
       ... 
8995     47
8996     41
8997     27
8998     51
8999     43
Length: 9000, dtype: int64

### Beta: diarios generalistas + especializados en deportes

In [19]:
# Columnas del dataframe de noticias que aplican a este grupo; las generales y las deportivas
cols_beta = [col for col in clientes_0 if (col.startswith('gen') or col.startswith('dep'))]

# Máscara para identificar las noticias que están disponibles en esta suscripción
mask_beta = []

for i in range(0, len(cols)):
    mask_beta.append(cols[i] in cols_beta)

# Máscara para identificar las noticias que NO están disponibles en esta suscripción
mask_no_beta = [not i for i in mask_beta]

# Las noticias que no están disponibles se quedan en nulo, para diferenciarlas de las que sí están disponibles pero no se han 
# leído (0)
clientes_beta[clientes_beta.columns[mask_no_beta]] = None

# Creamos dataframe de clientes_beta
cb = clientes_beta[clientes_beta.columns[mask_beta]].copy()

# Diferentes perfiles
cb.loc[9000:9999,] = np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[10000:10999,] = np.random.choice(np.arange(0, 2), p=[0.2, 0.8], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[11000:11999,] = np.random.choice(np.arange(0, 2), p=[0.3, 0.7], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[12000:12999,] = np.random.choice(np.arange(0, 2), p=[0.4, 0.6], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[13000:13999,] = np.random.choice(np.arange(0, 2), p=[0.5, 0.5], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[14000:14999,] = np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[15000:15999,] = np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[16000:16999,] = np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=(1000, len(clientes_beta.columns[mask_beta]))) 
cb.loc[17000:17999,] = np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=(1000, len(clientes_beta.columns[mask_beta]))) 

cb.insert(0, 'suscripcion', 'beta')

### Gamma: diarios generalistas + especializados en economía/finanzas

In [20]:
# Columnas del dataframe de noticias que aplican a este grupo; las generales y las de economía 
cols_gamma = [col for col in clientes_0 if (col.startswith('gen') or col.startswith('eco'))]

# Máscara para identificar las noticias que están disponibles en esta suscripción
mask_gamma = []

for i in range(0, len(cols)):
    mask_gamma.append(cols[i] in cols_gamma)

# Máscara para identificar las noticias que NO están disponibles en esta suscripción
mask_no_gamma = [not i for i in mask_gamma]

# Las noticias que no están disponibles se quedan en nulo, para diferenciarlas de las que sí están disponibles pero no se han 
# leído (0)
clientes_gamma[clientes_gamma.columns[mask_no_gamma]] = None

# Creamos dataframe de clientes_gamma
cg = clientes_gamma[clientes_gamma.columns[mask_gamma]].copy()

# Diferentes perfiles
cg.loc[18000:18999,] = np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[19000:19999,] = np.random.choice(np.arange(0, 2), p=[0.2, 0.8], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[20000:20999,] = np.random.choice(np.arange(0, 2), p=[0.3, 0.7], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[21000:21999,] = np.random.choice(np.arange(0, 2), p=[0.4, 0.6], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[22000:22999,] = np.random.choice(np.arange(0, 2), p=[0.5, 0.5], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[23000:23999,] = np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[24000:24999,] = np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[25000:25999,] = np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 
cg.loc[26000:26999,] = np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=(1000, len(clientes_gamma.columns[mask_gamma]))) 

cg.insert(0, 'suscripcion', 'gamma')

### Delta: suscripción "familiar", incluye la Omega (completa) x 4 usuarios

In [21]:
# Columnas del dataframe de noticias que aplican a este grupo; todas, pero lo identificamos así por si en un futuro 
# decidimos modificarlo
cols_delta = [col for col in clientes_0 if (col.startswith('gen') or col.startswith('eco') or col.startswith('dep'))]

# Máscara para identificar las noticias que están disponibles en esta suscripción
mask_delta = []

for i in range(0, len(cols)):
    mask_delta.append(cols[i] in cols_delta)

# Máscara para identificar las noticias que NO están disponibles en esta suscripción
mask_no_delta = [not i for i in mask_delta]

# Las noticias que no están disponibles se quedan en nulo, para diferenciarlas de las que sí están disponibles pero no se han 
# leído (0)
clientes_delta[clientes_delta.columns[mask_no_delta]] = None

# Creamos dataframe de clientes_delta
cd = clientes_delta[clientes_delta.columns[mask_delta]].copy()

# Diferentes perfiles
cd.loc[27000:27999,] = np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[28000:28999,] = np.random.choice(np.arange(0, 2), p=[0.2, 0.8], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[29000:29999,] = np.random.choice(np.arange(0, 2), p=[0.3, 0.7], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[30000:30999,] = np.random.choice(np.arange(0, 2), p=[0.4, 0.6], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[31000:31999,] = np.random.choice(np.arange(0, 2), p=[0.5, 0.5], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[32000:32999,] = np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[33000:33999,] = np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[34000:34999,] = np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=(1000, len(clientes_delta.columns[mask_delta]))) 
cd.loc[35000:35999,] = np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=(1000, len(clientes_delta.columns[mask_delta]))) 

cd.insert(0, 'suscripcion', 'delta')

### Omega: suscripción completa (diarios generalistas + deportes + economía/finanzas)

In [22]:
# Columnas del dataframe de noticias que aplican a este grupo; todas, pero lo identificamos así por si en un futuro 
# decidimos modificarlo
cols_omega = [col for col in clientes_0 if (col.startswith('gen') or col.startswith('eco') or col.startswith('dep'))]

# Máscara para identificar las noticias que están disponibles en esta suscripción
mask_omega = []

for i in range(0, len(cols)):
    mask_omega.append(cols[i] in cols_omega)

# Máscara para identificar las noticias que NO están disponibles en esta suscripción
mask_no_omega = [not i for i in mask_omega]

# Las noticias que no están disponibles se quedan en nulo, para diferenciarlas de las que sí están disponibles pero no se han 
# leído (0)
clientes_omega[clientes_omega.columns[mask_no_omega]] = None

# Creamos dataframe de clientes_omega
co = clientes_omega[clientes_omega.columns[mask_omega]].copy()

# Diferentes perfiles
co.loc[36000:36999,] = np.random.choice(np.arange(0, 2), p=[0.1, 0.9], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[37000:37999,] = np.random.choice(np.arange(0, 2), p=[0.2, 0.8], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[38000:38999,] = np.random.choice(np.arange(0, 2), p=[0.3, 0.7], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[39000:39999,] = np.random.choice(np.arange(0, 2), p=[0.4, 0.6], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[40000:40999,] = np.random.choice(np.arange(0, 2), p=[0.5, 0.5], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[41000:41999,] = np.random.choice(np.arange(0, 2), p=[0.6, 0.4], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[42000:42999,] = np.random.choice(np.arange(0, 2), p=[0.7, 0.3], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[43000:43999,] = np.random.choice(np.arange(0, 2), p=[0.8, 0.2], size=(1000, len(clientes_omega.columns[mask_omega]))) 
co.loc[44000:44999,] = np.random.choice(np.arange(0, 2), p=[0.9, 0.1], size=(1000, len(clientes_omega.columns[mask_omega]))) 

co.insert(0, 'suscripcion', 'omega')

## Dataframe conjunto

In [23]:
clientes = [ca, cb, cg, cd, co]
clientes_lectura = pd.concat(clientes).reset_index(drop=True)
clientes_lectura

,suscripcion,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,...,eco_860,eco_861,eco_862,eco_863,eco_864,eco_865,eco_866,eco_867,eco_868,eco_869
0,alpha,1,1,1,0,0,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alpha,1,1,1,1,1,0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alpha,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alpha,1,1,1,0,1,1,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alpha,1,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,omega,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
44996,omega,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
44997,omega,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44998,omega,0,1,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
# Exportar a csv
nombre = 'clientes_susc_lectura_' + datetime.today().strftime('%d%m%Y') + '.csv'
clientes_lectura.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')

### Dataframe de PUNTUACIONES

Asumimos que no todos los clientes que leen una noticia la puntúan (probabilidad de puntuación de noticia = 80%). Por tanto, sacamos otro dataframe con las puntuaciones de las noticias.

In [25]:
# Hacemos una copia del dataframe de clientes_lectura
clientes_rank = copy.deepcopy(clientes_lectura)

# Extraemos el tipo de suscripción de cada cliente
tipo_susc = clientes_rank.pop('suscripcion')

# Máscara que filtra cuáles sí han sido leídas
filtro_lectura = (clientes_rank == 1)

# A las noticias que NO han sido leídas le asignamos una puntuación de None (nulo)
clientes_rank[~filtro_lectura] = None

# Probabilidad de que un cliente puntúe una noticia tras haberla leído
p = 0.8

# Segundo filtro; asignamos valor nulo a aquellas noticias que sí han sido leídas pero no puntuadas
clientes_rank[filtro_lectura] = np.where(np.random.random(clientes_rank[filtro_lectura].shape) < (1-p), None, clientes_rank[filtro_lectura])

# Unificamos la nomenclatura de nulos; hasta aquí tenemos NaN y None. Pasamos todos los nulos a None
clientes_rank = clientes_rank.where(pd.notnull(clientes_rank), None)

# Tercer filtro: noticias que sí han sido puntuadas, tras haber filtrado las que han sido leídas pero no puntuadas
filtro_lectura2 = (clientes_rank == 1)

# De esas que han sido puntuadas, le asignamos una puntuación de 1 a 5 (como si fuesen estrellas? puntuación 1<2<3<4<5)
clientes_rank[filtro_lectura2] = np.random.randint(1,6, size=(45000, len(columnas)))

# Dataframe final con las puntuaciones
clientes_rank.insert(0, 'suscripcion', tipo_susc)
clientes_rank

,suscripcion,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,...,eco_860,eco_861,eco_862,eco_863,eco_864,eco_865,eco_866,eco_867,eco_868,eco_869
0,alpha,None,1,2,None,None,2,2,5,None,...,None,None,None,None,None,None,None,None,None,None
1,alpha,4,2,None,3,2,None,None,4,2,...,None,None,None,None,None,None,None,None,None,None
2,alpha,3,5,None,None,None,5,None,2,4,...,None,None,None,None,None,None,None,None,None,None
3,alpha,None,4,4,None,5,2,3,None,5,...,None,None,None,None,None,None,None,None,None,None
4,alpha,2,5,None,5,5,2,2,2,5,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,omega,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
44996,omega,1,3,None,None,None,None,None,None,None,...,None,None,None,None,None,4,None,None,None,None
44997,omega,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
44998,omega,None,1,None,None,None,None,None,None,None,...,3,None,None,4,None,None,None,None,None,None


In [26]:
# Cuántas noticias ha puntuado cada uno (se observa que es < al número de noticias leídas)
clientes_rank.count(axis=1)

0        306
1        321
2        324
3        336
4        312
        ... 
44995     73
44996     58
44997     66
44998     68
44999     69
Length: 45000, dtype: int64

In [27]:
# Exportar a csv
nombre = 'clientes_susc_rank_' + datetime.today().strftime('%d%m%Y') + '.csv'
clientes_rank.to_csv(nombre, encoding='utf-8-sig', header = True, sep = ';')

Por tanto, ya tendríamos dos dataframes de clientes:  
  
**clientes_lectura**: solo con valores 1 (si ha leído la noticia) y 0 (si no la ha leído).  
**clientes_rank**: con las puntuaciones que han dado a las noticias.